# Imports

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [ ]:
# Shared Imports
import random
import os
import pathlib
from typing import Iterator, List, Optional, Tuple
import json

import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image as pil_image
# model imports
## Pytorch/modell stuff
import torch
import torch.nn as nn
from torchmetrics import Accuracy
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Pre-processing
import albumentations as A
import albumentations.pytorch as APT
import cv2 
from tqdm import tqdm

# Globals

In [ ]:
SEED = 42
NUM_WORKERS = 2

# Wheter to PAD the images
PAD = True
# The size of the images
PATCH = (256, 256)
# The number of matches to consider
N_MATCHES = 5
# From trainning to ensure the same val-train division
VAL_SIZE = 0.1
BATCH_SIZE = 16
# The used embedding size
EMBEDDING_SIZE = 4096
# The used base model
BASE_MODEL = "efficientnet_b1"
# Set random Seed
pl.seed_everything(SEED)

In [ ]:
# directory of the data
DATA_DIR = pathlib.Path("../input/mlip-pad-resize-256x256")
# Work directory, where to store the data
WORKING_DIR = pathlib.Path("")
# Locations of the original train set, to derive the chain names
CHAIN_DIR = pathlib.Path("../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images")
# Locations of the train images in the data directory
TRAIN_DIR = DATA_DIR / pathlib.Path("pad_and_resize/train_images")
# Directory of the model weights and saved embeddings
MODEL_WEIGHTS_DIR = pathlib.Path("../input/mlip-hotelid-sim-weights-emb")
MODEL_WEIGHTS = pathlib.Path("../input/mlip-hotelid-sim-weights-emb/logs/lightning_logs/version_0/checkpoints/epoch_0009.step_000025139.val-map_6.4391.last.ckpt")
BASE_EMB = MODEL_WEIGHTS_DIR / "base_image-embeddings.pkl"

# Loading data

In [ ]:
hotel_id_code_df = pd.read_csv(MODEL_WEIGHTS_DIR / 'hotel_id_code_mapping.csv')
hotel_id_code_map = hotel_id_code_df.set_index('hotel_code').to_dict()["hotel_id"]
hotel_id_code_df.head()

In [ ]:
chain_names = os.listdir(CHAIN_DIR)
train_file = DATA_DIR / pathlib.Path("train.csv")

# Encode the chain identifiers so that the model can work with it and save it so it can be retrieved
train_df = pd.read_csv(train_file)    
train_df["hotel_code"] = train_df["hotel_id"].astype('category').cat.codes.values.astype(np.int64)

hotel_id_code_df = train_df.drop(columns=["image_id"]).drop_duplicates().reset_index(drop=True)
hotel_id_code_df.to_csv(WORKING_DIR / 'hotel_id_code_mapping.csv', index=False)

In [ ]:
print("Number of images:", len(train_df))
print("Number of different classes:", len(chain_names))
train_df.head()

In [ ]:
# Split the validation and train set
train_set, val_set = train_test_split(train_df, test_size=VAL_SIZE, random_state=SEED)
print("Number of train images:", len(train_set))
print("Number of val images:", len(val_set))
train_set.head()

## Pre-process

In [ ]:
def open_preprocess_img(img_path: pathlib.Path):
    img = cv2.imread(str(img_path))
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if PAD: img = pad(img)
    
    return cv2.resize(img, PATCH)

def pad(img):
    w, h, c = np.shape(img)
    const = 0
        
    if w == h: return img
    elif (w - h) % 2 != 0: const = 1
        
    if w < h:
        half_py = (h - w) // 2       
        return cv2.copyMakeBorder(img, 0, 0, half_py, half_py + const, cv2.BORDER_CONSTANT, value=0)
    elif h < w:
        half_px = (w - h) // 2
        return cv2.copyMakeBorder(img, half_px, half_px + const, 0, 0, cv2.BORDER_CONSTANT, value=0)

## Augmentations

In [ ]:
base_transform = A.Compose([
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])

val_aug = A.Compose([
    A.CoarseDropout(p=0.75, max_holes=1, 
                    min_height=PATCH[0]//4, max_height=PATCH[0]//2,
                    min_width=PATCH[1]//4,  max_width=PATCH[1]//2, 
                    fill_value=(255,0,0)),# simulating occlusions
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])

## Dataloader

In [ ]:
class ImageDataset(Dataset):
    def __init__(self,
                 data: pd.DataFrame,
                 data_path: pathlib.Path,
                 transform: Optional = None,
                ):
        self.data = data
        self.data_path = data_path
        self.transform = transform

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, idx: int):
        record = self.data.iloc[idx]

        image_path = self.data_path / record["image_id"]
        image = np.array(pil_image.open(image_path)).astype(np.uint8)
        
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed["image"]
        
        label = record['hotel_code']
        return image, label

# Model

## Arcface models

In [ ]:
# source: https://github.com/ronghuaiyang/arcface-pytorch/blob/master/models/metrics.py
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        output *= self.s

        return output

In [ ]:
class HotelModelArcface(pl.LightningModule):
    def __init__(self,
                n_hotels: int,
                steps_per_epoch: int,
                n_embeddings: int = 256,
                base_model = None,
                pretrained: bool = False,
                learning_rate: float = 0.003,
                
                ):
        super().__init__()
        
        # Hyperparams
        self.n_embeddings = n_embeddings
        self.n_hotels = n_hotels
        self.learning_rate = learning_rate
        self.steps_per_epoch = steps_per_epoch
        
        # Metrics
        self.loss_fn = nn.CrossEntropyLoss()
        self.train_acc = Accuracy()
        self.val_acc = Accuracy()
        
        # Model Definition 
        ## Base model
        self.base_model = timm.create_model(base_model, pretrained=False)        
        in_features = self.base_model.get_classifier().in_features
        
        fc_name, _ = list(self.base_model.named_modules())[-1]
        if fc_name == 'classifier':
            self.base_model.classifier = nn.Identity()
        elif fc_name == 'head.fc':
            self.base_model.head.fc = nn.Identity()
        elif fc_name == 'fc':
            self.base_model.fc = nn.Identity()
        else:
            raise Exception("unknown classifier layer: " + fc_name)
        
        ## Arcface module
        # self.arc_face = ArcMarginProduct(self.n_embeddings, n_hotels, s=30.0, m=0.20, easy_margin=False)
        
        ## Top model
        self.top_model = nn.Sequential(
            nn.utils.weight_norm(nn.Linear(in_features, self.n_embeddings*2), dim=None),
            nn.BatchNorm1d(self.n_embeddings*2),
            nn.Dropout(0.2),
            nn.utils.weight_norm(nn.Linear(self.n_embeddings*2, self.n_embeddings)),
            nn.BatchNorm1d(self.n_embeddings),
        )
        
        # Save hyper params
        self.save_hyperparameters()

    def configure_optimizers(self):
        optimizer = Lookahead(torch.optim.AdamW(self.parameters(), lr=self.learning_rate), k=3)
        
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
                    optimizer,
                    max_lr=self.learning_rate,
                    epochs=EPOCHS,
                    steps_per_epoch=self.steps_per_epoch,
                    div_factor=10,
                    final_div_factor=1,
                    pct_start=0.1,
                    anneal_strategy="cos",
                )
        
        schedule = {
            # Required: the scheduler instance.
            "scheduler": scheduler,
        }
        return [optimizer], [schedule]
    
    def forward(self, x, targets = None):
        y_hat = self.base_model(x)
        y_hat = y_hat.view(y_hat.size(0), -1)
        y_hat = self.top_model(y_hat)
        
        if targets is not None:
            y_hat = self.arc_face(y_hat, targets)

        return y_hat

    def training_step(self, batch, batch_idx):
        x, y = batch
        
        # Forward pass
        y_hat = self.forward(x, y)
        loss = self.loss_fn(y_hat, y)
        self.train_acc(y_hat, y)

        # Store results
        self.log("train_loss", loss, prog_bar=False)
        
        return loss
    
    def training_epoch_end(self, train_step_outputs) -> None:
        # Log metrics
        self.log("train_acc", self.train_acc, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        
        # Forward pass
        y_hat = self.forward(x, y)
        loss = self.loss_fn(y_hat, y)
        self.val_acc(y_hat, y)

        # Store results
        self.log("val_loss", loss, prog_bar=False)
        return y_hat
        
    def validation_epoch_end(self, validation_step_outputs) -> None:
        self.log("val_acc", self.val_acc, prog_bar=True)
        
    def predict_step(self, batch, batch_idx):
        y_hat = self.forward(batch)
        return y_hat
    
    def test_step(self, batch, batch_idx):
        # Forward pass
        x, y = batch
        sample_emb = self.forward(x)
        
        # Cosine sim
        cosine_sim = F.cosine_similarity(sample_emb, self.base_emb)
        sorted_idx = torch.argsort(cosine_sim, descending=True)
        sorted_lbls = self.base_labels[sorted_idx]

        # Because unique does not return properly
        top_lbls = []
        i = 0
        while len(top_lbls) < 5:
            if sorted_lbls[i] not in top_lbls:
                top_lbls.append(sorted_lbls[i])
            i += 1
        top_lbls = torch.Tensor(top_lbls).long().to(('cuda' if torch.cuda.is_available() else 'cpu'))
        
        # Calculate MAP@5
        y_idx = (y == top_lbls).nonzero(as_tuple=True)[0]
        if len(y_idx) == 0:
            map5 = torch.zeros(1).to(('cuda' if torch.cuda.is_available() else 'cpu'))
        else:
            map5 = 1 / (y_idx+1)

        acc_top_1 = (y == sorted_lbls[0])
        acc_top_5 = (y == top_lbls[:N_MATCHES])
        return map5, acc_top_1.float(), acc_top_5.float()
        
    def test_epoch_end(self, test_step_outputs) -> None:
        map5 = torch.stack([ap for ap, _, _ in test_step_outputs])
        acc_top_1 = torch.stack([acc1 for _, acc1, _ in test_step_outputs])
        acc_top_5 = torch.stack([acc5 for _, _, acc5 in test_step_outputs])
        print(len(map5), len(acc_top_1), len(acc_top_5))
        print(torch.mean(map5))
        print(torch.mean(acc_top_1))
        print(torch.mean(acc_top_5.any(axis=1).float()))
        
        self.log("train_map5", torch.mean(map5), prog_bar=True)
        self.log("train_acc", torch.mean(acc_top_1), prog_bar=True)
        self.log("train-acc5", torch.mean(acc_top_5.any(axis=1).float()), prog_bar=True)
        
    def set_base_emb(self, base_emb, base_labels):
        self.base_emb = base_emb
        self.base_labels = base_labels

# Base model

In [ ]:
class HotelModel(pl.LightningModule):
    def __init__(self,
                n_hotels: int,
                steps_per_epoch: int,
                n_embeddings: int = 256,
                base_model = None,
                pretrained: bool = False,
                learning_rate: float = 0.003,
                
                ):
        super().__init__()
        
        # Hyperparams
        self.n_embeddings = n_embeddings
        self.n_hotels = n_hotels
        self.learning_rate = learning_rate
        self.steps_per_epoch = steps_per_epoch
        
        # Metrics
        # self.loss_fn = F.cross_entropy
        self.train_acc = Accuracy()
        self.val_acc = Accuracy()
        
        # Model Definition 
        ## Base model
        self.base_model = timm.create_model(base_model, pretrained=False)        
        in_features = self.base_model.get_classifier().in_features
        
        fc_name, _ = list(self.base_model.named_modules())[-1]
        if fc_name == 'classifier':
            self.base_model.classifier = nn.Identity()
        elif fc_name == 'head.fc':
            self.base_model.head.fc = nn.Identity()
        elif fc_name == 'fc':
            self.base_model.fc = nn.Identity()
        else:
            raise Exception("unknown classifier layer: " + fc_name)
        
        ## Top model
        self.top_model = nn.Sequential(
            nn.utils.weight_norm(nn.Linear(in_features, self.n_embeddings*2), dim=None),
            nn.BatchNorm1d(self.n_embeddings*2),
            nn.Dropout(0.2),
            nn.utils.weight_norm(nn.Linear(self.n_embeddings*2, self.n_embeddings)),
        )
        
        ## Classifier
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(self.n_embeddings),
            nn.Dropout(0.2),
            nn.Linear(self.n_embeddings, self.n_hotels),
        )
        
        # Save hyper params
        self.save_hyperparameters()

    def configure_optimizers(self):
        optimizer = Lookahead(torch.optim.AdamW(self.parameters(), lr=self.learning_rate), k=3)
        
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
                    optimizer,
                    max_lr=self.learning_rate,
                    epochs=EPOCHS,
                    steps_per_epoch=self.steps_per_epoch,
                    div_factor=10,
                    final_div_factor=1,
                    pct_start=0.1,
                    anneal_strategy="cos",
                )
        
        schedule = {
            # Required: the scheduler instance.
            "scheduler": scheduler,
        }
        return [optimizer], [schedule]
    
    def forward(self, x):
        emb = self.base_model(x)
        emb = emb.view(emb.size(0), -1)
        emb = self.top_model(emb)
        y_hat = self.classifier(emb)
        return emb

    def test_step(self, batch, batch_idx):
        # Forward pass
        x, y = batch
        sample_emb = self.forward(x)
        
        # Cosine sim
        cosine_sim = F.cosine_similarity(sample_emb, self.base_emb)
        sorted_idx = torch.argsort(cosine_sim, descending=True)
        sorted_lbls = self.base_labels[sorted_idx]

        # Because unique does not return properly
        top_lbls = []
        i = 0
        while len(top_lbls) < 5:
            if sorted_lbls[i] not in top_lbls:
                top_lbls.append(sorted_lbls[i])
            i += 1
        top_lbls = torch.Tensor(top_lbls).long().to(('cuda' if torch.cuda.is_available() else 'cpu'))
        
        # Calculate MAP@5
        y_idx = (y == top_lbls).nonzero(as_tuple=True)[0]
        if len(y_idx) == 0:
            map5 = torch.zeros(1).to(('cuda' if torch.cuda.is_available() else 'cpu'))
        else:
            map5 = 1 / (y_idx+1)

        acc_top_1 = (y == sorted_lbls[0])
        acc_top_5 = (y == top_lbls[:N_MATCHES])
        return map5, acc_top_1.float(), acc_top_5.float()
        
    def test_epoch_end(self, test_step_outputs) -> None:
        map5 = torch.stack([ap for ap, _, _ in test_step_outputs])
        acc_top_1 = torch.stack([acc1 for _, acc1, _ in test_step_outputs])
        acc_top_5 = torch.stack([acc5 for _, _, acc5 in test_step_outputs])
        print(len(map5), len(acc_top_1), len(acc_top_5))
        print(torch.mean(map5))
        print(torch.mean(acc_top_1))
        print(torch.mean(acc_top_5.any(axis=1).float()))
        
        self.log("train_map5", torch.mean(map5), prog_bar=True)
        self.log("train_acc", torch.mean(acc_top_1), prog_bar=True)
        self.log("train-acc5", torch.mean(acc_top_5.any(axis=1).float()), prog_bar=True)
        
    def set_base_emb(self, base_emb, base_labels):
        self.base_emb = base_emb
        self.base_labels = base_labels

## Loaders for the model

In [ ]:
def get_arc_model(model_type, backbone_name, checkpoint_path, args):
    model = HotelModelArcface.load_from_checkpoint(checkpoint_path, map_location='cpu', strict=False)
    return model

In [ ]:
def get_model(model_type, backbone_name, checkpoint_path, args):
    model = HotelModel.load_from_checkpoint(checkpoint_path, map_location='cpu', strict=False)
    return model

## Helper Functions

In [ ]:
def generate_torch_embeddings(loader, model, bar_desc="Generating embeds"):
    targets_all = []
    outputs_all = []
    
    model = model.to(args.device)
    model.eval()
    with torch.no_grad():
        x = tqdm(loader, desc=bar_desc)
        for i, sample in enumerate(x):
            input = sample[0].to(args.device)
            target = sample[1].to(args.device)
            output = model(input)
            
            targets_all.extend(target.cpu().numpy())
            outputs_all.extend(output.detach().cpu())

    targets_all = np.array(targets_all)
    print(len(outputs_all))
    outputs_all = torch.stack(outputs_all)
    print(outputs_all.shape)
            
    return outputs_all, targets_all

# Generate the embeddings and test the model

In [ ]:
class args:
    epochs = 5
    lr = 1e-3
    batch_size = 64
    num_workers = 2
    val_samples = 1
    embedding_size = 128
    backbone_name = "efficientnet_b0"
    n_classes = train_df["hotel_code"].nunique()
    device = ('cuda' if torch.cuda.is_available() else 'cpu')
    
pattern = "epoch_{epoch:04d}.step_{step:09d}.val-map_{val_loss:.4f}"
ModelCheckpoint.CHECKPOINT_NAME_LAST = pattern + ".last"
checkpointer = ModelCheckpoint(
        monitor="val_acc",
        filename=pattern + ".best",
        save_last=True,
        auto_insert_metric_name=False,
        save_top_k=1,
    )

trainer = pl.Trainer(
    max_epochs=1,
    gpus=torch.cuda.device_count(),
    callbacks=[checkpointer, LearningRateMonitor()],
    default_root_dir="logs/",
)

In [ ]:
train_data = ImageDataset(train_set, TRAIN_DIR, transform=base_transform)
val_data = ImageDataset(val_set, TRAIN_DIR, transform=val_aug)

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, drop_last=True)
val_dataloader = DataLoader(val_data, batch_size=1, shuffle=False, num_workers=NUM_WORKERS)

In [ ]:
model = get_arc_model("classification", 
                  BASE_MODEL,
                  MODEL_WEIGHTS, 
                  args)

base_embeds, base_targets = generate_torch_embeddings(train_dataloader, model, "Generate base embeddings")

model.set_base_emb(base_embeds.to("cuda"), torch.from_numpy(base_targets).to("cuda"))

trainer.test(model, dataloaders=val_dataloader)

# Model specific

## Arcface

In [ ]:
MODEL_WEIGHTS = pathlib.Path("../input/mlip-hotelid-sim-weights-emb/logs/lightning_logs/version_0/checkpoints/epoch_0009.step_000025139.val-map_6.4391.last.ckpt")

model = get_arc_model("classification", 
                  BASE_MODEL,
                  MODEL_WEIGHTS, 
                  args)

base_embeds, base_targets = generate_torch_embeddings(train_dataloader, model, "Generate base embeddings")

model.set_base_emb(base_embeds.to("cuda"), torch.from_numpy(base_targets).to("cuda"))

trainer.test(model, dataloaders=val_dataloader)

## Cosface

In [ ]:
MODEL_WEIGHTS = pathlib.Path("../input/cosfaceweights/logs/lightning_logs/version_0/checkpoints/epoch_0009.step_000025139.val-acc_0.0002.last.ckpt")

model = get_model("classification", 
                  BASE_MODEL,
                  MODEL_WEIGHTS, 
                  args)

base_embeds, base_targets = generate_torch_embeddings(train_dataloader, model, "Generate base embeddings")

model.set_base_emb(base_embeds.to("cuda"), torch.from_numpy(base_targets).to("cuda"))

trainer.test(model, dataloaders=val_dataloader)

## Tripletloss

In [ ]:
MODEL_WEIGHTS = pathlib.Path("../input/tripletloss-weights/logs/lightning_logs/version_0/checkpoints/epoch_0009.step_000025139.val-map_0.1461.last.ckpt")

model = get_model("classification", 
                  BASE_MODEL,
                  MODEL_WEIGHTS, 
                  args)

base_embeds, base_targets = generate_torch_embeddings(train_dataloader, model, "Generate base embeddings")

model.set_base_emb(base_embeds.to("cuda"), torch.from_numpy(base_targets).to("cuda"))

trainer.test(model, dataloaders=val_dataloader)

## Crossent

In [ ]:
MODEL_WEIGHTS = pathlib.Path("../input/crossent-weights/logs/lightning_logs/version_0/checkpoints/epoch_0009.step_000025139.val-map_2.7154.last.ckpt")

model = get_model("classification", 
                  BASE_MODEL,
                  MODEL_WEIGHTS, 
                  args)

base_embeds, base_targets = generate_torch_embeddings(train_dataloader, model, "Generate base embeddings")

model.set_base_emb(base_embeds.to("cuda"), torch.from_numpy(base_targets).to("cuda"))

trainer.test(model, dataloaders=val_dataloader)